<a href="https://colab.research.google.com/github/hiroto-noguchi/weekly_articles_2023/blob/main/weekly_article_2023_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
#事前学習モデルで「あの。」につづく文を生成する。
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelWithLMHead.from_pretrained("rinna/japanese-gpt2-medium")
input = tokenizer.encode("あの。", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=5)
for result in tokenizer.batch_decode(output):
  print(result)

In [ ]:
#すべてのファイルを読み込み、テキストを結合する。
import os
text_train = ''
file_list = os.listdir('/content/')
for file in file_list:
  if file.startswith('KSJ') and file.endswith('.txt'):
    f = open(file)
    text_train = text_train + f.read()
    f.close()
text_train[:1000]

In [ ]:
#協力者が複数行にわたって発言している部分のみを抽出する。
text = text_train.split('*i: ')
extracted_text = ''
for line in text:
  temp = line.strip().split('\n')
  if len(temp) >= 3:
    cont = ''.join(temp[1:])
    cont = cont.replace('*s: ','')
    cont = cont.replace(' ','')
    extracted_text = extracted_text + cont + '\n'
extracted_text[:1000]

In [ ]:
#ファイルに保存する
f = open('train.txt', 'w')
f.write(extracted_text)
f.close()

In [ ]:
#GPUの使用を確認する。
!nvidia-smi

In [ ]:
!git clone https://github.com/huggingface/transformers -b v4.23.1
!pip install evaluate

In [ ]:
#ファインチューニングする。
%%time
!python ./transformers/examples/pytorch/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --validation_file=train.txt \
    --do_train \
    --do_eval \
    --num_train_epochs=3 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --per_device_train_batch_size=1 \
    --per_device_eval_batch_size=1 \
    --output_dir=output/

In [ ]:
#ファインチューニングしたモデルを確かめる。
from transformers import AutoModelForCausalLM
model_rev = AutoModelForCausalLM.from_pretrained("output/")

input = tokenizer.encode("あの。", return_tensors="pt")
output = model_rev.generate(input, do_sample=True, max_length=100, num_return_sequences=5)
for result in tokenizer.batch_decode(output):
  print(result)